In [1]:
import pandas as pd
import numpy as np 
import os
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_absolute_error, classification_report, accuracy_score
from xgboost import XGBClassifier
from sklearn.grid_search import GridSearchCV

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
os.chdir("ETL Legislative 2T/data")

In [3]:
ls

 Le volume dans le lecteur D s'appelle Data
 Le num‚ro de s‚rie du volume est 44FE-89D6

 R‚pertoire de D:\Documents\Projet_fil_rouge\Elections\legislatives\ETL Legislative 2T\data

07/06/2017  13:40    <DIR>          .
07/06/2017  13:40    <DIR>          ..
03/06/2017  13:50           185ÿ019 cdsp_legi1997t2_circ.xlsx
03/06/2017  13:50           470ÿ016 cdsp_legi2002t2_circ.xls
03/06/2017  13:50           293ÿ376 cdsp_legi2007t2_circ.xls
03/06/2017  13:50           163ÿ234 cdsp_legi2012t2_circ.xlsx
03/06/2017  13:50        12ÿ499ÿ195 dataframe_elections.xlsx
07/06/2017  13:40           805ÿ969 dataset_legislative_2nd_prediction.csv
06/06/2017  23:16           569ÿ871 pred_format_second_tour_du_6_6_21h46.xlsx
               7 fichier(s)       14ÿ986ÿ680 octets
               2 R‚p(s)  951ÿ587ÿ491ÿ840 octets libres


In [4]:
df = pd.read_csv("dataset_legislative_2nd_prediction.csv")
df["elu"] = df["elu"].map(lambda x: 1 if x=='O' else 0)

In [5]:
df.head()

,an,département,circonscription,code,name,first_name,bloc,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,...,top_triangulaire,bloc_opposant_Centre,bloc_opposant_Divers,bloc_opposant_Droite,bloc_opposant_Exdroite,bloc_opposant_Exgauche,bloc_opposant_Gauche,voix_bloc_1nd,voix_bloc_2nd_lag,voix_candidat_2nd_lag
0,2012,AIN,1.0,1|1,BRETON,XAVIER,Droite,1.0,1.0,0.2570,...,0,0,0,0,0,0,1,0.2241,0.567573,0.536621
1,2012,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,Gauche,0.0,0.0,0.2147,...,0,0,0,1,0,0,0,0.2316,0.432427,0.463379
2,2012,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,Droite,1.0,1.0,0.2753,...,1,0,0,0,1,0,1,0.2209,0.567573,NaN
3,2012,AIN,2.0,1|2,EYRAUD,OLIVIER,Exdroite,0.0,0.0,0.1798,...,1,0,0,1,0,0,1,0.1252,NaN,NaN
4,2012,AIN,2.0,1|2,RAYMOND,MICHEL,Gauche,0.0,0.0,0.2027,...,1,0,0,1,1,0,0,0.1368,0.432427,NaN


In [6]:
df.columns

Index(['an', 'département', 'circonscription', 'code', 'name', 'first_name',
       'bloc', 'circo_leg_meme_nuance', 'circo_pres_meme_nuance',
       'score_bloc_pres', 'depute_sortant', 'ancien_depute', 'au_gouvernement',
       'ancien_ministre', 'p_voix', 'p_voix_candidat_2nd', 'elu',
       'top_triangulaire', 'bloc_opposant_Centre', 'bloc_opposant_Divers',
       'bloc_opposant_Droite', 'bloc_opposant_Exdroite',
       'bloc_opposant_Exgauche', 'bloc_opposant_Gauche', 'voix_bloc_1nd',
       'voix_bloc_2nd_lag', 'voix_candidat_2nd_lag'],
      dtype='object')

In [7]:
nd_tour = pd.get_dummies(pd.qcut(df["voix_bloc_2nd_lag"], 5, labels=False), prefix="bloc_2nd_tour" )
nd_tour["bloc_2nd_tour_nan"] = nd_tour.apply(lambda x : 0 if np.sum(x)==1 else 1 , axis=1)

nd_tour_candidate = pd.get_dummies(pd.qcut(df["voix_candidat_2nd_lag"], 5, labels=False), 
                                   prefix="2nd_tour_candidate" )
nd_tour_candidate["2nd_tour_candidate_nan"] = nd_tour_candidate.apply(lambda x : 0 if np.sum(x)==1 else 1 , axis=1)

nd = pd.concat([nd_tour, nd_tour_candidate], axis=1)

In [8]:
df = pd.concat([df, nd], axis=1)

In [9]:
df.columns

Index(['an', 'département', 'circonscription', 'code', 'name', 'first_name',
       'bloc', 'circo_leg_meme_nuance', 'circo_pres_meme_nuance',
       'score_bloc_pres', 'depute_sortant', 'ancien_depute', 'au_gouvernement',
       'ancien_ministre', 'p_voix', 'p_voix_candidat_2nd', 'elu',
       'top_triangulaire', 'bloc_opposant_Centre', 'bloc_opposant_Divers',
       'bloc_opposant_Droite', 'bloc_opposant_Exdroite',
       'bloc_opposant_Exgauche', 'bloc_opposant_Gauche', 'voix_bloc_1nd',
       'voix_bloc_2nd_lag', 'voix_candidat_2nd_lag', 'bloc_2nd_tour_0.0',
       'bloc_2nd_tour_1.0', 'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0',
       'bloc_2nd_tour_4.0', 'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0',
       '2nd_tour_candidate_1.0', '2nd_tour_candidate_2.0',
       '2nd_tour_candidate_3.0', '2nd_tour_candidate_4.0',
       '2nd_tour_candidate_nan'],
      dtype='object')

In [10]:
del df['voix_candidat_2nd_lag']
del df['voix_bloc_2nd_lag']

In [11]:
df = pd.get_dummies(df, columns=["bloc"], prefix="bloc")

In [12]:
df.columns

Index(['an', 'département', 'circonscription', 'code', 'name', 'first_name',
       'circo_leg_meme_nuance', 'circo_pres_meme_nuance', 'score_bloc_pres',
       'depute_sortant', 'ancien_depute', 'au_gouvernement', 'ancien_ministre',
       'p_voix', 'p_voix_candidat_2nd', 'elu', 'top_triangulaire',
       'bloc_opposant_Centre', 'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche', 'voix_bloc_1nd', 'bloc_2nd_tour_0.0',
       'bloc_2nd_tour_1.0', 'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0',
       'bloc_2nd_tour_4.0', 'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0',
       '2nd_tour_candidate_1.0', '2nd_tour_candidate_2.0',
       '2nd_tour_candidate_3.0', '2nd_tour_candidate_4.0',
       '2nd_tour_candidate_nan', 'bloc_Centre', 'bloc_Divers', 'bloc_Droite',
       'bloc_Exdroite', 'bloc_Exgauche', 'bloc_Gauche'],
      dtype='object')

In [13]:
df.head()

,an,département,circonscription,code,name,first_name,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,...,2nd_tour_candidate_2.0,2nd_tour_candidate_3.0,2nd_tour_candidate_4.0,2nd_tour_candidate_nan,bloc_Centre,bloc_Divers,bloc_Droite,bloc_Exdroite,bloc_Exgauche,bloc_Gauche
0,2012,AIN,1.0,1|1,BRETON,XAVIER,1.0,1.0,0.2570,1.0,...,1,0,0,0,0,0,1,0,0,0
1,2012,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,0.0,0.0,0.2147,0.0,...,0,0,0,0,0,0,0,0,0,1
2,2012,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,1.0,1.0,0.2753,1.0,...,0,0,0,1,0,0,1,0,0,0
3,2012,AIN,2.0,1|2,EYRAUD,OLIVIER,0.0,0.0,0.1798,0.0,...,0,0,0,1,0,0,0,1,0,0
4,2012,AIN,2.0,1|2,RAYMOND,MICHEL,0.0,0.0,0.2027,0.0,...,0,0,0,1,0,0,0,0,0,1


In [14]:
df = df[[
        'an', 'département','circonscription','code', 'name','first_name',
       'circo_leg_meme_nuance', 'circo_pres_meme_nuance', 'score_bloc_pres',
       'depute_sortant', 'ancien_depute', 'au_gouvernement', 'ancien_ministre',
        
        'bloc_opposant_Centre', 'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche',
        
        'bloc_2nd_tour_0.0',
       'bloc_2nd_tour_1.0', 'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0',
       'bloc_2nd_tour_4.0', 'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0',
       '2nd_tour_candidate_1.0', '2nd_tour_candidate_2.0',
       '2nd_tour_candidate_3.0', '2nd_tour_candidate_4.0',
       '2nd_tour_candidate_nan', 
    
        
        'bloc_Centre', 'bloc_Divers', 'bloc_Droite',
       'bloc_Exdroite', 'bloc_Exgauche', 'bloc_Gauche',
    
       'p_voix', 'voix_bloc_1nd', 'top_triangulaire', 'p_voix_candidat_2nd', 'elu']]
        


In [15]:
df.head()

,an,département,circonscription,code,name,first_name,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,...,bloc_Divers,bloc_Droite,bloc_Exdroite,bloc_Exgauche,bloc_Gauche,p_voix,voix_bloc_1nd,top_triangulaire,p_voix_candidat_2nd,elu
0,2012,AIN,1.0,1|1,BRETON,XAVIER,1.0,1.0,0.2570,1.0,...,0,1,0,0,0,0.2241,0.2241,0,0.515715,1
1,2012,AIN,1.0,1|1,DEBAT,JEAN-FRANCOIS,0.0,0.0,0.2147,0.0,...,0,0,0,0,1,0.2316,0.2316,0,0.484285,0
2,2012,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,1.0,1.0,0.2753,1.0,...,0,1,0,0,0,0.2209,0.2209,1,0.443146,1
3,2012,AIN,2.0,1|2,EYRAUD,OLIVIER,0.0,0.0,0.1798,0.0,...,0,0,1,0,0,0.1252,0.1252,1,0.169303,0
4,2012,AIN,2.0,1|2,RAYMOND,MICHEL,0.0,0.0,0.2027,0.0,...,0,0,0,0,1,0.1368,0.1368,1,0.387551,0


In [17]:
#X_train = df[(df["an"]!=2012)&(df["an"]!=1997)]
#X_test = df[df["an"]==2012]
#y_train = df.ix[(df["an"]!=2012)&(df["an"]!=1997), "score"]
#y_test = df.ix[df["an"]==2012, "score"]
#y_train_cl = df.ix[(df["an"]!=2012)&(df["an"]!=1997), "elu"]
#y_test_cl = df.ix[df["an"]==2012, "elu"]

In [18]:
#del X_train["score"]
#del X_test["score"]
#del X_train["elu"]
#del X_test["elu"]

In [19]:
#del X_train["an"]
#del X_test["code"]
#del X_train["code"]
#del X_test["an"]

In [20]:
#X_train.columns

In [21]:
#rl = LinearRegression()
#rl.fit(X_train[['voix_bloc_1nd',  'top_triangulaire']], y_train)

In [22]:
#rl.score(X_test[['voix_bloc_1nd',  'top_triangulaire']], y_test)

In [23]:
#rf = RandomForestRegressor()
#rf.fit(X_train, y_train)

In [24]:
#rf.score(X_test, y_test)

In [25]:
#svm = SVR(kernel="rbf",C=15, gamma=0.2)
#svm.fit(X_train, y_train)

In [26]:
#svm.score(X_test, y_test)

In [27]:
#mean_absolute_error(y_test, svm.predict(X_test))

In [21]:
X_train = df[(df["an"]!=2012)&(df["an"]!=1997)&(df["an"]!=2017)]
X_test = df[df["an"]==2012]
y_train_cl = df.ix[(df["an"]!=2012)&(df["an"]!=1997)&(df["an"]!=2017), "elu"]
y_test_cl = df.ix[df["an"]==2012, "elu"]
X_validation = df[(df["an"]==2017)]
del X_train['p_voix_candidat_2nd']
del X_test['p_voix_candidat_2nd']
del X_train["elu"]
del X_test["elu"]
del X_train["an"]
del X_test["an"]
del X_train["code"]
del X_train["département"]
del X_test["département"]
del X_train["name"]
del X_test["name"]
del X_train['first_name']
del X_test['first_name']
del X_train['circonscription']
del X_test['circonscription']

In [22]:
X_test.columns

Index(['code', 'circo_leg_meme_nuance', 'circo_pres_meme_nuance',
       'score_bloc_pres', 'depute_sortant', 'ancien_depute', 'au_gouvernement',
       'ancien_ministre', 'bloc_opposant_Centre', 'bloc_opposant_Divers',
       'bloc_opposant_Droite', 'bloc_opposant_Exdroite',
       'bloc_opposant_Exgauche', 'bloc_opposant_Gauche', 'bloc_2nd_tour_0.0',
       'bloc_2nd_tour_1.0', 'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0',
       'bloc_2nd_tour_4.0', 'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0',
       '2nd_tour_candidate_1.0', '2nd_tour_candidate_2.0',
       '2nd_tour_candidate_3.0', '2nd_tour_candidate_4.0',
       '2nd_tour_candidate_nan', 'bloc_Centre', 'bloc_Divers', 'bloc_Droite',
       'bloc_Exdroite', 'bloc_Exgauche', 'bloc_Gauche', 'p_voix',
       'voix_bloc_1nd', 'top_triangulaire'],
      dtype='object')

In [23]:
X_train.columns

Index(['circo_leg_meme_nuance', 'circo_pres_meme_nuance', 'score_bloc_pres',
       'depute_sortant', 'ancien_depute', 'au_gouvernement', 'ancien_ministre',
       'bloc_opposant_Centre', 'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche', 'bloc_2nd_tour_0.0', 'bloc_2nd_tour_1.0',
       'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0', 'bloc_2nd_tour_4.0',
       'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0', '2nd_tour_candidate_1.0',
       '2nd_tour_candidate_2.0', '2nd_tour_candidate_3.0',
       '2nd_tour_candidate_4.0', '2nd_tour_candidate_nan', 'bloc_Centre',
       'bloc_Divers', 'bloc_Droite', 'bloc_Exdroite', 'bloc_Exgauche',
       'bloc_Gauche', 'p_voix', 'voix_bloc_1nd', 'top_triangulaire'],
      dtype='object')

In [24]:
X_train.head()

,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,ancien_depute,au_gouvernement,ancien_ministre,bloc_opposant_Centre,bloc_opposant_Divers,bloc_opposant_Droite,...,2nd_tour_candidate_nan,bloc_Centre,bloc_Divers,bloc_Droite,bloc_Exdroite,bloc_Exgauche,bloc_Gauche,p_voix,voix_bloc_1nd,top_triangulaire
1099,1.0,1.0,0.3110,0.0,0.0,0.0,0.0,0,0,0,...,1,0,0,1,0,0,0,0.2799,0.2799,0
1100,0.0,0.0,0.2214,0.0,0.0,0.0,0.0,0,0,1,...,1,0,0,0,0,0,1,0.1895,0.1895,0
1101,1.0,1.0,0.2974,1.0,1.0,0.0,0.0,0,0,0,...,0,0,0,1,0,0,0,0.2736,0.2736,0
1102,0.0,0.0,0.2122,0.0,0.0,0.0,0.0,0,0,1,...,1,0,0,0,0,0,1,0.1040,0.1040,0
1103,0.0,1.0,0.2554,0.0,0.0,0.0,0.0,0,0,0,...,1,0,0,1,0,0,0,0.1783,0.1783,0


In [25]:
clf1 = SVC(probability=True)
clf2 = RandomForestClassifier()
clf3= XGBClassifier(min_child_weight=2, n_estimators=400, 
                    learning_rate=0.02, max_depth=3)

clf1.fit(X_train, y_train_cl)
clf2.fit(X_train, y_train_cl)
clf3.fit(X_train, y_train_cl)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.02, max_delta_step=0, max_depth=3,
       min_child_weight=2, missing=None, n_estimators=400, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [26]:
predict_1 = clf1.predict_proba(X_test.drop(["code"], axis=1))
predict_2 = clf2.predict_proba(X_test.drop(["code"], axis=1))
predict_3 = clf3.predict_proba(X_test.drop(["code"], axis=1))

In [27]:
clf1.classes_

array([0, 1])

In [28]:
df_1 = pd.Series( (v[1] for v in predict_1) , name="svc")
df_2 = pd.Series( (v[1] for v in predict_2) , name="rf")
df_3 = pd.Series( (v[1] for v in predict_3) , name="xgb")

In [29]:
X_test = X_test.reset_index(drop=True)

In [30]:
X_test = pd.concat([X_test, df_1, df_2, df_3], axis=1)

In [31]:
X_test

,code,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,ancien_depute,au_gouvernement,ancien_ministre,bloc_opposant_Centre,bloc_opposant_Divers,...,bloc_Droite,bloc_Exdroite,bloc_Exgauche,bloc_Gauche,p_voix,voix_bloc_1nd,top_triangulaire,svc,rf,xgb
0,1|1,1.0,1.0,0.2570,1.0,1.0,0.0,0.0,0,0,...,1,0,0,0,0.2241,0.2241,0,0.899404,0.2,0.535904
1,1|1,0.0,0.0,0.2147,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0.2316,0.2316,0,0.128163,0.3,0.180060
2,1|2,1.0,1.0,0.2753,1.0,1.0,0.0,0.0,0,0,...,1,0,0,0,0.2209,0.2209,1,0.965826,0.8,0.816438
3,1|2,0.0,0.0,0.1798,0.0,0.0,0.0,0.0,0,0,...,0,1,0,0,0.1252,0.1252,1,0.041169,0.3,0.050229
4,1|2,0.0,0.0,0.2027,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0.1368,0.1368,1,0.268590,0.1,0.070581
5,1|3,1.0,1.0,0.2671,1.0,1.0,0.0,0.0,0,0,...,1,0,0,0,0.2214,0.2214,0,0.955686,0.9,0.778302
6,1|3,0.0,0.0,0.2090,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0.1742,0.1742,0,0.169498,0.1,0.093517
7,1|4,0.0,0.0,0.1944,0.0,0.0,0.0,0.0,0,0,...,0,0,0,1,0.1823,0.1823,0,0.172156,0.0,0.098948
8,1|4,1.0,1.0,0.2876,1.0,1.0,0.0,0.0,0,0,...,1,0,0,0,0.2370,0.2370,0,0.946488,0.4,0.852537
9,1|5,1.0,1.0,0.2412,0.0,0.0,0.0,0.0,0,0,...,1,0,0,0,0.1795,0.1795,0,0.849302,0.5,0.505396


In [32]:
X_test["mean"] = X_test.apply(lambda x: np.max([x["svc"], x["xgb"], x["rf"]]), axis=1)
X_test["max"] = X_test.apply(lambda x: np.mean([x["svc"], x["xgb"], x["rf"]]), axis=1)

In [33]:
X_test.shape

(1101, 40)

In [34]:
X_test = pd.merge(X_test, X_test.groupby(["code"])["rf"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["svc"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["xgb"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["mean"].max().to_frame().reset_index(), how="left", on="code")
X_test = pd.merge(X_test, X_test.groupby(["code"])["max"].max().to_frame().reset_index(), how="left", on="code")

In [35]:
X_test.head()

,code,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,ancien_depute,au_gouvernement,ancien_ministre,bloc_opposant_Centre,bloc_opposant_Divers,...,svc_x,rf_x,xgb_x,mean_x,max_x,rf_y,svc_y,xgb_y,mean_y,max_y
0,1|1,1.0,1.0,0.2570,1.0,1.0,0.0,0.0,0,0,...,0.899404,0.2,0.535904,0.899404,0.545103,0.3,0.899404,0.535904,0.899404,0.545103
1,1|1,0.0,0.0,0.2147,0.0,0.0,0.0,0.0,0,0,...,0.128163,0.3,0.180060,0.300000,0.202741,0.3,0.899404,0.535904,0.899404,0.545103
2,1|2,1.0,1.0,0.2753,1.0,1.0,0.0,0.0,0,0,...,0.965826,0.8,0.816438,0.965826,0.860754,0.8,0.965826,0.816438,0.965826,0.860754
3,1|2,0.0,0.0,0.1798,0.0,0.0,0.0,0.0,0,0,...,0.041169,0.3,0.050229,0.300000,0.130466,0.8,0.965826,0.816438,0.965826,0.860754
4,1|2,0.0,0.0,0.2027,0.0,0.0,0.0,0.0,0,0,...,0.268590,0.1,0.070581,0.268590,0.146391,0.8,0.965826,0.816438,0.965826,0.860754


In [36]:
X_test["label_svc"] = X_test.apply(lambda x: 0 if x["svc_x"]<x["svc_y"] else 1, axis=1)
X_test["label_rf"] = X_test.apply(lambda x: 0 if x["rf_x"]<x["rf_y"] else 1, axis=1)
X_test["label_xgb"] = X_test.apply(lambda x: 0 if x["xgb_x"]<x["xgb_y"] else 1, axis=1)
X_test["label_mean"] = X_test.apply(lambda x: 0 if x["mean_x"]<x["mean_y"] else 1, axis=1)
X_test["label_max"] = X_test.apply(lambda x: 0 if x["max_x"]<x["max_y"] else 1, axis=1)

In [37]:
X_test

,code,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,ancien_depute,au_gouvernement,ancien_ministre,bloc_opposant_Centre,bloc_opposant_Divers,...,rf_y,svc_y,xgb_y,mean_y,max_y,label_svc,label_rf,label_xgb,label_mean,label_max
0,1|1,1.0,1.0,0.2570,1.0,1.0,0.0,0.0,0,0,...,0.3,0.899404,0.535904,0.899404,0.545103,1,0,1,1,1
1,1|1,0.0,0.0,0.2147,0.0,0.0,0.0,0.0,0,0,...,0.3,0.899404,0.535904,0.899404,0.545103,0,1,0,0,0
2,1|2,1.0,1.0,0.2753,1.0,1.0,0.0,0.0,0,0,...,0.8,0.965826,0.816438,0.965826,0.860754,1,1,1,1,1
3,1|2,0.0,0.0,0.1798,0.0,0.0,0.0,0.0,0,0,...,0.8,0.965826,0.816438,0.965826,0.860754,0,0,0,0,0
4,1|2,0.0,0.0,0.2027,0.0,0.0,0.0,0.0,0,0,...,0.8,0.965826,0.816438,0.965826,0.860754,0,0,0,0,0
5,1|3,1.0,1.0,0.2671,1.0,1.0,0.0,0.0,0,0,...,0.9,0.955686,0.778302,0.955686,0.877996,1,1,1,1,1
6,1|3,0.0,0.0,0.2090,0.0,0.0,0.0,0.0,0,0,...,0.9,0.955686,0.778302,0.955686,0.877996,0,0,0,0,0
7,1|4,0.0,0.0,0.1944,0.0,0.0,0.0,0.0,0,0,...,0.4,0.946488,0.852537,0.946488,0.733008,0,0,0,0,0
8,1|4,1.0,1.0,0.2876,1.0,1.0,0.0,0.0,0,0,...,0.4,0.946488,0.852537,0.946488,0.733008,1,1,1,1,1
9,1|5,1.0,1.0,0.2412,0.0,0.0,0.0,0.0,0,0,...,0.5,0.849302,0.505396,0.849302,0.618233,1,1,1,1,1


In [38]:
print(classification_report(X_test["label_svc"], y_test_cl))
print(accuracy_score(X_test["label_svc"], y_test_cl))

             precision    recall  f1-score   support

          0       0.81      0.81      0.81       560
          1       0.80      0.80      0.80       541

avg / total       0.80      0.80      0.80      1101

0.80199818347


In [39]:
print(classification_report(X_test["label_rf"], y_test_cl))
print(accuracy_score(X_test["label_rf"], y_test_cl))

             precision    recall  f1-score   support

          0       0.77      0.83      0.80       520
          1       0.84      0.78      0.81       581

avg / total       0.80      0.80      0.80      1101

0.80199818347


In [40]:
print(classification_report(X_test["label_xgb"], y_test_cl))
print(accuracy_score(X_test["label_xgb"], y_test_cl))

             precision    recall  f1-score   support

          0       0.82      0.82      0.82       558
          1       0.82      0.82      0.82       543

avg / total       0.82      0.82      0.82      1101

0.820163487738


In [41]:
print(classification_report(X_test["label_mean"], y_test_cl))
print(accuracy_score(X_test["label_mean"], y_test_cl))

             precision    recall  f1-score   support

          0       0.82      0.82      0.82       559
          1       0.81      0.81      0.81       542

avg / total       0.82      0.82      0.82      1101

0.815622161671


In [42]:
print(classification_report(X_test["label_max"], y_test_cl))
print(accuracy_score(X_test["label_max"], y_test_cl))

             precision    recall  f1-score   support

          0       0.81      0.81      0.81       560
          1       0.81      0.81      0.81       541

avg / total       0.81      0.81      0.81      1101

0.809264305177


In [200]:
gbm = XGBClassifier()
gbm_params = {
'learning_rate': [0.01,0.02],
'n_estimators': [300,350,400],
'max_depth': [3,4],
"min_child_weight":[2]
}


grid = GridSearchCV(gbm, gbm_params,verbose=10,n_jobs=-1)
grid.fit(X_train, y_train_cl)
print (grid.best_params_)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=3 
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=3 
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=3 
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=3 
[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=3, score=0.795455 -   1.8s
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=3 
[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=3, score=0.803030 -   1.8s
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=3 
[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=3, score=0.797872 -   2.2s
[CV] min_child_weight=2, n_estimators=400, learning_rate=0.01, max_depth=3 
[CV]  min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=3, score=0.806061 -   2.6s
[CV

[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:    4.1s


[CV]  min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=3, score=0.792424 -   2.5s
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=4 
[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.01, max_depth=3, score=0.809091 -   2.7s
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=4 
[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.01, max_depth=3, score=0.790909 -   2.5s
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=4 
[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.01, max_depth=3, score=0.793313 -   2.7s
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=4 
[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=4, score=0.813636 -   2.5s
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=4 


[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    6.9s


[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=4, score=0.795455 -   2.6s
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=4 
[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.01, max_depth=4, score=0.794833 -   2.4s
[CV] min_child_weight=2, n_estimators=400, learning_rate=0.01, max_depth=4 
[CV]  min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=4, score=0.815152 -   2.6s
[CV] min_child_weight=2, n_estimators=400, learning_rate=0.01, max_depth=4 
[CV]  min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=4, score=0.800000 -   2.9s
[CV] min_child_weight=2, n_estimators=400, learning_rate=0.01, max_depth=4 
[CV]  min_child_weight=2, n_estimators=350, learning_rate=0.01, max_depth=4, score=0.793313 -   3.7s
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.02, max_depth=3 
[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.01, max_depth=4, score=0.810606 -   3.8s
[CV] min_child

...

[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.02, max_depth=3, score=0.818182 -   2.5s
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.02, max_depth=4 
[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.02, max_depth=3, score=0.806061 -   2.4s
[CV] min_child_weight=2, n_estimators=300, learning_rate=0.02, max_depth=4 
[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.02, max_depth=3, score=0.796353 -   2.6s
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.02, max_depth=4 
[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.02, max_depth=4, score=0.810606 -   2.0s
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.02, max_depth=4 
[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.02, max_depth=4, score=0.787234 -   1.4s
[CV] min_child_weight=2, n_estimators=350, learning_rate=0.02, max_depth=4 
[CV]  min_child_weight=2, n_estimators=300, learning_rate=0.02, max_depth=4, score=0.803030 -   2.0s
[CV] min_child

[Parallel(n_jobs=-1)]: Done  33 out of  36 | elapsed:   21.0s remaining:    1.9s


[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.02, max_depth=4, score=0.807576 -   2.9s
[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.02, max_depth=4, score=0.791793 -   2.6s
[CV]  min_child_weight=2, n_estimators=400, learning_rate=0.02, max_depth=4, score=0.800000 -   2.7s


[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed:   23.5s finished


{'min_child_weight': 2, 'n_estimators': 400, 'learning_rate': 0.02, 'max_depth': 3}


In [202]:
X_test.columns

Index(['circo_leg_meme_nuance', 'code', 'circo_pres_meme_nuance',
       'score_bloc_pres', 'depute_sortant', 'ancien_depute', 'au_gouvernement',
       'ancien_ministre', 'top_triangulaire', 'bloc_opposant_Centre',
       'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche', 'voix_bloc_1nd', 'bloc_Centre', 'bloc_Divers',
       'bloc_Droite', 'bloc_Exdroite', 'bloc_Exgauche', 'bloc_Gauche', 'svc_x',
       'rf_x', 'xgb_x', 'mean_x', 'max_x', 'rf_y', 'svc_y', 'xgb_y', 'mean_y',
       'max_y', 'label_svc', 'label_rf', 'label_xgb', 'label_mean',
       'label_max'],
      dtype='object')

In [16]:
clf_f = XGBClassifier(min_child_weight=2, n_estimators=400, 
                    learning_rate=0.02, max_depth=3)


X_train = df[(df["an"]!=1997)&(df["an"]!=2017)]
y_train_cl = df.ix[(df["an"]!=1997)&(df["an"]!=2017), "elu"]
X_validation = df[(df["an"]==2017)]
del X_train['p_voix_candidat_2nd']
del X_train["elu"]
del X_train["an"]
del X_train["département"]
del X_train["code"]
del X_train["name"]
del X_train["first_name"]
del X_train['circonscription']

del X_validation["an"]
del X_validation["p_voix_candidat_2nd"]
del X_validation['elu']

In [17]:
X_train.columns

Index(['circo_leg_meme_nuance', 'circo_pres_meme_nuance', 'score_bloc_pres',
       'depute_sortant', 'ancien_depute', 'au_gouvernement', 'ancien_ministre',
       'bloc_opposant_Centre', 'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche', 'bloc_2nd_tour_0.0', 'bloc_2nd_tour_1.0',
       'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0', 'bloc_2nd_tour_4.0',
       'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0', '2nd_tour_candidate_1.0',
       '2nd_tour_candidate_2.0', '2nd_tour_candidate_3.0',
       '2nd_tour_candidate_4.0', '2nd_tour_candidate_nan', 'bloc_Centre',
       'bloc_Divers', 'bloc_Droite', 'bloc_Exdroite', 'bloc_Exgauche',
       'bloc_Gauche', 'p_voix', 'voix_bloc_1nd', 'top_triangulaire'],
      dtype='object')

In [18]:
X_validation.columns

Index(['département', 'circonscription', 'code', 'name', 'first_name',
       'circo_leg_meme_nuance', 'circo_pres_meme_nuance', 'score_bloc_pres',
       'depute_sortant', 'ancien_depute', 'au_gouvernement', 'ancien_ministre',
       'bloc_opposant_Centre', 'bloc_opposant_Divers', 'bloc_opposant_Droite',
       'bloc_opposant_Exdroite', 'bloc_opposant_Exgauche',
       'bloc_opposant_Gauche', 'bloc_2nd_tour_0.0', 'bloc_2nd_tour_1.0',
       'bloc_2nd_tour_2.0', 'bloc_2nd_tour_3.0', 'bloc_2nd_tour_4.0',
       'bloc_2nd_tour_nan', '2nd_tour_candidate_0.0', '2nd_tour_candidate_1.0',
       '2nd_tour_candidate_2.0', '2nd_tour_candidate_3.0',
       '2nd_tour_candidate_4.0', '2nd_tour_candidate_nan', 'bloc_Centre',
       'bloc_Divers', 'bloc_Droite', 'bloc_Exdroite', 'bloc_Exgauche',
       'bloc_Gauche', 'p_voix', 'voix_bloc_1nd', 'top_triangulaire'],
      dtype='object')

In [19]:
clf_f.fit(X_train, y_train_cl)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.02, max_delta_step=0, max_depth=3,
       min_child_weight=2, missing=None, n_estimators=400, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [20]:
predict = clf_f.predict_proba(X_validation.drop(["département", 'circonscription',"code","name","first_name"], axis=1))

In [21]:
df_prob = pd.Series( (v[1] for v in predict) , name="predict_proba")
X_validation = X_validation.reset_index(drop=True)
X_validation = pd.concat([X_validation, df_prob], axis=1)

In [22]:
X_validation = pd.merge(X_validation, X_validation.groupby(["code"])["predict_proba"].max().to_frame().reset_index(), how="left", on="code")

In [23]:
X_validation["elu"] = X_validation.apply(lambda x: 0 if x["predict_proba_x"]<x["predict_proba_y"] else 1, axis=1)

In [24]:
X_validation.head()

,département,circonscription,code,name,first_name,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,ancien_depute,...,bloc_Droite,bloc_Exdroite,bloc_Exgauche,bloc_Gauche,p_voix,voix_bloc_1nd,top_triangulaire,predict_proba_x,predict_proba_y,elu
0,AIN,1.0,1|1,BRETON,XAVIER,1.0,0.0,0.347790,1.0,1.0,...,1,0,0,0,0.330423,0.330423,0,0.961737,0.961737,1
1,AIN,1.0,1|1,MALLET,LAURENT,0.0,0.0,0.285034,0.0,0.0,...,0,0,0,0,0.277258,0.277258,0,0.769219,0.961737,0
2,AIN,2.0,1|2,BEGUET,MARIE-JEANNE,0.0,0.0,0.304912,0.0,0.0,...,0,0,0,0,0.264182,0.264182,0,0.951293,0.976836,0
3,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,1.0,0.0,0.354690,1.0,1.0,...,1,0,0,0,0.326547,0.326547,0,0.976836,0.976836,1
4,AIN,3.0,1|3,GIVERNET,OLGA,0.0,1.0,0.315591,0.0,0.0,...,0,0,0,0,0.292552,0.292552,0,0.960362,0.978534,0


In [25]:
te = X_validation.groupby(["code"])["elu"].sum().reset_index()
te[te["elu"]>1]

,code,elu
561,987|3,2


In [26]:
X_validation[X_validation["code"]=="987|3"]

,département,circonscription,code,name,first_name,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,ancien_depute,...,bloc_Droite,bloc_Exdroite,bloc_Exgauche,bloc_Gauche,p_voix,voix_bloc_1nd,top_triangulaire,predict_proba_x,predict_proba_y,elu
851,POLYNESIE-FRANCAISE,3.0,987|3,DUBOIS,VINCENT,1.0,0.0,0.233448,0.0,0.0,...,1,0,0,0,0.268376,0.536752,0,0.680865,0.680865,1
852,POLYNESIE-FRANCAISE,3.0,987|3,HOWELL,PATRICK,1.0,0.0,0.233448,0.0,0.0,...,1,0,0,0,0.268376,0.536752,0,0.680865,0.680865,1


In [27]:
df_col = pd.read_excel("dataframe_elections.xlsx")
df_col = df_col[df_col["an"]==2017]
df_col  = df_col.rename(columns={"nom":"name","prenom":"first_name"})
df_col.head()

,an,c_dep,dep,circo,code,inscrits,geo_frontalier,geo_dom,geo_idf,etrangers,...,sexe,name,first_name,etiquette,nuance,nuance_groupe,bloc,voix,p_voix,second_tour
0,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,F,BLATRIX-CONTAT,FLORENCE,NaN,SOC,SOC,Gauche,NaN,NaN,NaN
1,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,M,BONNOT,GILBERT,NaN,DIV,DIV,Divers,NaN,NaN,NaN
2,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,M,BRETON,XAVIER,NaN,LR,LR,Droite,NaN,NaN,NaN
3,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,M,BUISSON,JEROME,NaN,FN,FN,Exdroite,NaN,NaN,NaN
4,2017,1,AIN,1,1|1,82653.0,1,0,0,0.0793,...,F,CARLIER,MARIE,NaN,DIV,DIV,Divers,NaN,NaN,NaN


In [28]:
# add partie politique
X_validation_df = pd.merge(X_validation, df_col[["code","name","first_name","nuance_groupe"]], 
                     on=["code","name","first_name"],how="left")

In [29]:
X_validation_df.head()

,département,circonscription,code,name,first_name,circo_leg_meme_nuance,circo_pres_meme_nuance,score_bloc_pres,depute_sortant,ancien_depute,...,bloc_Exdroite,bloc_Exgauche,bloc_Gauche,p_voix,voix_bloc_1nd,top_triangulaire,predict_proba_x,predict_proba_y,elu,nuance_groupe
0,AIN,1.0,1|1,BRETON,XAVIER,1.0,0.0,0.347790,1.0,1.0,...,0,0,0,0.330423,0.330423,0,0.961737,0.961737,1,LR
1,AIN,1.0,1|1,MALLET,LAURENT,0.0,0.0,0.285034,0.0,0.0,...,0,0,0,0.277258,0.277258,0,0.769219,0.961737,0,MDM
2,AIN,2.0,1|2,BEGUET,MARIE-JEANNE,0.0,0.0,0.304912,0.0,0.0,...,0,0,0,0.264182,0.264182,0,0.951293,0.976836,0,MDM
3,AIN,2.0,1|2,DE LA VERPILLIERE,CHARLES,1.0,0.0,0.354690,1.0,1.0,...,0,0,0,0.326547,0.326547,0,0.976836,0.976836,1,LR
4,AIN,3.0,1|3,GIVERNET,OLGA,0.0,1.0,0.315591,0.0,0.0,...,0,0,0,0.292552,0.292552,0,0.960362,0.978534,0,REM


In [39]:
exportCsv = X_validation_df[['département','circonscription','code','first_name','name','nuance_groupe','elu']]


exportCsv['code'] = exportCsv['code'].apply(str.lower)

exportCsv['code'] = exportCsv['code'].apply(str).apply(lambda x : x.replace('|', '-'))

exportCsv['candidat'] = exportCsv['first_name'].apply(str.capitalize) + ' ' + exportCsv['name'].apply(str.capitalize)

exportCsv['nom circo'] = exportCsv['département'].apply(

    lambda x : x.replace('-', ' ')).apply(str.title) + ' - ' + exportCsv['circonscription'].apply(str)

exportCsv = exportCsv.drop(['département', 'circonscription', 'first_name', 'name'], axis=1)


exportCsv = exportCsv.sort_values(['code', 'elu'], ascending=[True, False])

#print(exportCsv1.head())



circos = exportCsv['code'].unique()

temp = pd.DataFrame()

d = {}

for c in circos:

    temp = exportCsv[exportCsv['code'] == c]

    l = []

    for index, rows in temp.iterrows():

        l.extend(rows[1:].values)

    d[c] = l



    

duels = {k: v for k, v in d.items() if len(v) < 11}

triangulaires = {k: v for k, v in d.items() if len(v) > 10}



duels = pd.DataFrame(duels)

duels.index = ['color1','score1','candidat1','nom circo','color2','score2','candidat2','nom circo']

duels = duels.T

duels = duels.iloc[:,:7]


triangulaires = pd.DataFrame(triangulaires)

triangulaires.index = ['color1','score1','candidat1','nom circo','color2','score2','candidat2','nom circo','color3','score3','candidat3','nom circo']

triangulaires = triangulaires.T

nomCirco = triangulaires['nom circo'].iloc[:,0]

triangulaires = triangulaires.drop('nom circo', axis=1)

triangulaires['nom circo'] = nomCirco



final = pd.concat([duels, triangulaires])

final = final.fillna(value='')



final.index.name = 'circo'

print(final.head())

final.to_csv('resultats2.csv', index=True)

/home/brehelin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/brehelin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/brehelin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/

ValueError: arrays must all be same length